# PointNet Part Segmentation

## Imports

In [14]:
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import open3d as o3d

from pathlib import Path
import os

In [15]:
if (torch.cuda.is_available()):
    print(torch.cuda.device_count(), torch.cuda.current_device())

## Visualizing Data

In [46]:
eg_pts_hash = '1d1b37ce6d72d7855096c0dd2594842a'
eg_pts_path = f'data/train/pts/{eg_pts_hash}.pts'
eg_labels_path = f'data/train/label/{eg_pts_hash}.txt'

ex_pc = np.loadtxt(eg_pts_path, delimiter=' ')
ex_labels = np.loadtxt(eg_labels_path, delimiter=' ', dtype=np.int8)

print(ex_pc.shape, ex_labels.shape)

(2704, 3) (2704,)


In [72]:
# Uncomment to run, requires system GUI
visualize(torch.from_numpy(ex_pc), torch.from_numpy(ex_labels))

tensor([[ 0.2174, -0.1176, -0.0927],
        [ 0.2398, -0.2393,  0.1549],
        [-0.1579, -0.3512, -0.1093],
        ...,
        [-0.0696, -0.2838, -0.0418],
        [ 0.0372, -0.2923,  0.1277],
        [ 0.0550, -0.3040,  0.1593]], dtype=torch.float64) tensor([3, 3, 3,  ..., 3, 3, 3], dtype=torch.int8)
True [[ 0.21737 -0.11761 -0.09266]
 [ 0.23978 -0.23933  0.15489]
 [-0.15789 -0.3512  -0.10932]
 ...
 [-0.06959 -0.28383 -0.04181]
 [ 0.03722 -0.29234  0.12771]
 [ 0.05504 -0.30405  0.15925]] [3 3 3 ... 3 3 3]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


## Building the PointNet

In [94]:
class ChairCloudDataset(Dataset):
    def __init__(self, root, train=True, transform=torch.from_numpy, target_transform=torch.from_numpy):
        self.train = train

        self.transform = transform
        self.target_transform = target_transform

        root = Path(root)
        pts_dir = None
        labels_dir = None
        if self.train:
            pts_dir = root / Path('pts')
            labels_dir = root / Path('label')
        else:
            pts_dir = Path(root)

        self.pts_paths = []
        self.label_paths = []
        for pts_file in os.listdir(pts_dir):
            self.pts_paths.append(pts_dir / pts_file)

            if self.train:
                self.label_paths.append(labels_dir / (Path(pts_file).stem + '.txt'))

        dataset_type = 'train' if self.train else 'test'
        print(f'Found {len(self.pts_paths)} {dataset_type} datapoints')

    def __getitem__(self, index):
        pts_file = self.pts_paths[index]
        pts = np.loadtxt(pts_file, delimiter=' ')
        pts = self.transform(pts)

        if self.train:
            label_file = self.label_paths[index]
            seg = np.loadtxt(label_file, delimiter=' ', dtype=np.uint8)
            seg = self.target_transform(seg)

            return pts, seg
        
        return pts

    def __len__(self):
        return len(self.pts_paths)
            



In [95]:
train_dataset = ChairCloudDataset('data/train', train=True)
test_dataset = ChairCloudDataset('data/test', train=False)

Found 1000 train datapoints
Found 6 test datapoints


In [96]:
print('train first datapoint:', train_dataset[0])
print('test first datapoint:', test_dataset[0])

train first datapoint: (tensor([[ 0.2444, -0.1462, -0.1531],
        [ 0.1605, -0.1110,  0.1341],
        [-0.0194, -0.1618, -0.1123],
        ...,
        [-0.0416, -0.1171, -0.0834],
        [-0.0768, -0.0417, -0.0886],
        [ 0.1969,  0.0032,  0.1800]], dtype=torch.float64), tensor([3, 4, 4,  ..., 2, 2, 3], dtype=torch.uint8))
test first datapoint: tensor([[ 0.2209, -0.3227, -0.2592],
        [-0.1419,  0.0443,  0.2039],
        [-0.0833,  0.0780, -0.1899],
        ...,
        [ 0.1731, -0.2647,  0.2755],
        [-0.0665, -0.1003,  0.1920],
        [-0.0629, -0.0916, -0.1633]], dtype=torch.float64)


In [99]:
def visualize(pts, labels=None):

    is_labeled = False
    if type(labels) != type(None):
        is_labeled = True

    try:
        pts = pts.numpy()
        labels = labels.numpy()
        print('Tensor converted to numpy')
    except:
        print('Passed numpy')

    # red, green, blue, purple
    colors = np.array([[255,0,0],[0,255,0],[0,0,255],[100,0,100]])
    if is_labeled: labels = colors[labels-1]

    print(pts.shape, labels.shape)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)
    if is_labeled: pcd.colors = o3d.utility.Vector3dVector(labels)

    o3d.visualization.draw_geometries([pcd])

In [100]:
# Uncomment to visualize
pts, seg = train_dataset[0]
visualize(pts, seg)

<class 'numpy.ndarray'>
(2736, 3) (2736, 3)
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
